In [7]:
import json
import pandas as pd
jsonFile_snli = "chaosNLI_v1.0/chaosNLI_snli.jsonl"
jsonFile_mnli = "chaosNLI_v1.0/chaosNLI_mnli_m.jsonl"
jsonFile_anli = "chaosNLI_v1.0/chaosNLI_alphanli.jsonl"

In [8]:
def flatten_nested_json_df(df):

    df = df.reset_index()

    print(f"original shape: {df.shape}")
    print(f"original columns: {df.columns}")


    # search for columns to explode/flatten
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()

    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()

    print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []

        for col in dict_columns:
            print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace

#         for col in list_columns:
#             print(f"exploding: {col}")
#             # explode lists vertically, adding new columns
#             df = df.drop(columns=[col]).join(df[col].explode().to_frame())
#             new_columns.append(col)

        # check if there are still dict o list fields to flatten
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()

        print(f"lists: {list_columns}, dicts: {dict_columns}")

    print(f"final shape: {df.shape}")
    print(f"final columns: {df.columns}")
    return df

In [9]:
snli = flatten_nested_json_df(pd.read_json(jsonFile_snli, lines=True))
mnli = flatten_nested_json_df(pd.read_json(jsonFile_mnli, lines=True))
# anli = flatten_nested_json_df(pd.read_json(jsonFile_anli, lines=True))

original shape: (1514, 10)
original columns: Index(['index', 'uid', 'label_counter', 'majority_label', 'label_dist',
       'label_count', 'entropy', 'example', 'old_label', 'old_labels'],
      dtype='object')
lists: ['label_dist', 'label_count', 'old_labels'], dicts: ['label_counter', 'example']
flattening: label_counter
flattening: example
lists: [], dicts: []
final shape: (1514, 15)
final columns: Index(['index', 'uid', 'majority_label', 'label_dist', 'label_count',
       'entropy', 'old_label', 'old_labels', 'label_counter.n',
       'label_counter.e', 'label_counter.c', 'example.uid', 'example.premise',
       'example.hypothesis', 'example.source'],
      dtype='object')
original shape: (1599, 10)
original columns: Index(['index', 'uid', 'label_counter', 'majority_label', 'label_dist',
       'label_count', 'entropy', 'example', 'old_label', 'old_labels'],
      dtype='object')
lists: ['label_dist', 'label_count', 'old_labels'], dicts: ['label_counter', 'example']
flattening: l

In [10]:
name = ['snli', 'mnli']
for nli in [snli, mnli]:
    print("mean of neutral: ", nli['label_counter.n'].mean())
    print("mean of contradiction: ", nli['label_counter.c'].mean())
    print("mean of entailment: ", nli['label_counter.e'].mean())

mean of neutral:  51.05973154362416
mean of contradiction:  24.788998357963877
mean of entailment:  35.477987421383645
mean of neutral:  39.30100125156446
mean of contradiction:  22.091154625253207
mean of entailment:  41.696891191709845


In [5]:
def inspect(clause, nli):
    for index, data in nli[clause].iterrows():
        print("[n, c, e]:", [data['label_counter.{t}'.format(t=c)] for c in ['n', 'c', 'e']])
        print("premise:", data['example.premise'])
        print("hypothesis:", data['example.hypothesis'])
        
# def look(s):
#     for index, data in s.iterrows():
#         print("[n, c, e]:", [data['label_counter.{t}'.format(t=c)] for c in ['n', 'c', 'e']])
#         print("premise:", data['example.premise'])
#         print("hypothesis:", data['example.hypothesis'])

## Filtering by labels
- Are data points with mostly neutral labels good data points?

Most of these data points are trivially arguable. 

Garbage data point:

**premise**: A dog holding a Frisbee walking in the water.

**hypothesis**: The frisbee is orange.

**comment**: Of course it can be orange, but it is trivially arguable that it can be any other color. Question, do we need to include this kind of data points into training? This can still be meaningful


#### Might be a good example

**Context**: A man is reading a newspaper in a car dealership.

**Question**: Is it true that "The man is waiting for his car to be serviced."? 

**Answer**: Not necessarily. 

**Explanation**: The man could be waiting for his car to be serviced, or he could be waiting for something else entirely. For example, he could be waiting for a friend, or for a salesperson to approach him. Without more information, it's difficult to say for sure what the man is doing.

In [104]:
inspect(snli['label_counter.n'] <= 20, snli)

[n, c, e]: [2.0, nan, 98.0]
premise: Two young boys of opposing teams play football, while wearing full protection uniforms and helmets.
hypothesis: boys play football
[n, c, e]: [1.0, 94.0, 5.0]
premise: Families waiting in line at an amusement park for their turn to ride.
hypothesis: People are waiting in line at a restaurant.
[n, c, e]: [18.0, nan, 82.0]
premise: A man in a green jersey and rollerskates stumbles as a man in a black jersey appears to collide with him.
hypothesis: The man stumbles stumbles in rollerskates.
[n, c, e]: [18.0, 1.0, 81.0]
premise: A person powerwashing stairs leading down to a pool.
hypothesis: a worker cleans near the pool
[n, c, e]: [9.0, 2.0, 89.0]
premise: Two black little boys hanging over an upside down bicycle, one examining the pedal.
hypothesis: Children were looking at a bike.
[n, c, e]: [16.0, 82.0, 2.0]
premise: Two black little boys hanging over an upside down bicycle, one examining the pedal.
hypothesis: Someone is riding a bike.
[n, c, e]: 

[n, c, e]: [14.0, 1.0, 85.0]
premise: Three women on a stage, one wearing red shoes, black pants, and a gray shirt is sitting on a prop, another is sitting on the floor, and the third wearing a black shirt and pants is standing, as a gentleman in the back tunes an instrument.
hypothesis: The man tuning the instrument is near three women
[n, c, e]: [4.0, 1.0, 95.0]
premise: A large elderly woman in a green stripy dress is carrying shopping bags along a cobbled street.
hypothesis: a woman in green walks along a street
[n, c, e]: [9.0, 3.0, 88.0]
premise: A man pulls a bicycle-driven cart through a city alley.
hypothesis: A man is in an urban location.
[n, c, e]: [12.0, 1.0, 87.0]
premise: A small group of adults and children are in a room with large glass windows and using rolling pins.
hypothesis: A small group of people are gathered to use rolling pins.
[n, c, e]: [4.0, 3.0, 93.0]
premise: A man drags a young boy in swim trunks into the surf at what appears to be a beach in a tropical 

## Guess: Probably data points with a lot of entailments are good data points

However, most of these data points are merely summary

e.g.

**premise**: An old man is sitting in a chair outside a colorfully painted building.

**hypothesis**: A man is sitting on a chair

**premise**: Cyclist participating in an organized event cross a large suspension bridge and two women riders have stopped to pose for a picture.

**hypothesis**: some cyclists cross a bridge

In [55]:
inspect(snli['label_counter.e'] >= 80, snli)

[n, c, e]: [2.0, nan, 98.0]
premise: Two young boys of opposing teams play football, while wearing full protection uniforms and helmets.
hypothesis: boys play football
[n, c, e]: [nan, 2.0, 98.0]
premise: A man in a blue shirt standing in front of a garage-like structure painted with geometric designs.
hypothesis: A man is wearing a blue shirt
[n, c, e]: [nan, nan, 100.0]
premise: Under a blue sky with white clouds, a child reaches up to touch the propeller of a plane standing parked on a field of grass.
hypothesis: A child is reaching to touch the propeller of a plane.
[n, c, e]: [18.0, nan, 82.0]
premise: A man in a green jersey and rollerskates stumbles as a man in a black jersey appears to collide with him.
hypothesis: The man stumbles stumbles in rollerskates.
[n, c, e]: [18.0, 1.0, 81.0]
premise: A person powerwashing stairs leading down to a pool.
hypothesis: a worker cleans near the pool
[n, c, e]: [9.0, 2.0, 89.0]
premise: Two black little boys hanging over an upside down bicy

## What happens with data points with a lot of controversies
- Most of them are bs

In [103]:
inspect(snli['label_counter.c'] >= 70, snli)

[n, c, e]: [1.0, 94.0, 5.0]
premise: Families waiting in line at an amusement park for their turn to ride.
hypothesis: People are waiting in line at a restaurant.
[n, c, e]: [16.0, 82.0, 2.0]
premise: Two black little boys hanging over an upside down bicycle, one examining the pedal.
hypothesis: Someone is riding a bike.
[n, c, e]: [3.0, 95.0, 2.0]
premise: two people are waiting for a train at a station.
hypothesis: They are both waiting for a bus.
[n, c, e]: [23.0, 76.0, 1.0]
premise: A gentlemen dressed in a purple shirt with a red apron selling items to eat at a sport event held during the day in the summer time.
hypothesis: There is a person selling food inside a fancy restaurant.
[n, c, e]: [16.0, 83.0, 1.0]
premise: A cyclist in neon yellow uniform rides past spectators behind a yellow banner reading "Lotos."
hypothesis: A cyclist is taking a water break.
[n, c, e]: [4.0, 96.0, nan]
premise: A man dressed in bright colored clothing is sitting on a chair on the sidewalk playing a

## Not Too Bad
**Context**: A young child sits on some pillows with a green tray of chopped-up food in front of him.

**Question**: Is it true that "A young child prepares to eat a meal"? 

**Answer**: Not necessarily. 

**Explanation**: The child in the picture could be preparing to eat a meal, but it is also possible that he or she has already eaten a meal and is now playing with the food, or that the child is not interested in eating the food and is just sitting in front of it. Without more information, it's difficult to say for sure what is happening in the picture.

In [102]:
inspect((snli['label_counter.c'] >= 15) & (snli['label_counter.c'] <= 50) & \
       (snli['label_counter.n'] >= 15) & (snli['label_counter.n'] <= 50) & 
       (snli['label_counter.e'] >= 15) & (snli['label_counter.e'] <= 50), snli)

[n, c, e]: [36.0, 15.0, 49.0]
premise: At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.
hypothesis: A crowd is in front of a camera
[n, c, e]: [33.0, 31.0, 36.0]
premise: Number 13 kicks a soccer ball towards the goal during children's soccer game.
hypothesis: A player passing the ball in a soccer game.
[n, c, e]: [26.0, 32.0, 42.0]
premise: A photographer snaps a midair action shot of a snowboarder.
hypothesis: The midair shot snaps at a boarder of snow.
[n, c, e]: [32.0, 24.0, 44.0]
premise: Three women, two wearing red shirts and one wearing a purple shirt, and a man, wearing a light blue shirt, jump on a basketball court with balls in their hands.
hypothesis: Three people's are playing basketball and holds and ball in their hands
[n, c, e]: [49.0, 16.0, 35.0]
premise: A man in a red shirt is sitting at his desk with one leg on the desk, using a computer with two screens set up side by side.
hypot

In [13]:
inspect((snli['label_counter.c'] - snli['label_counter.e'] <= 10) & (snli['label_counter.e'] - snli['label_counter.c'] <= 10) & \
       (snli['label_counter.n'] <= 20), snli)

[n, c, e]: [10.0, 40.0, 50.0]
premise: A man with a striped shirt looks at a load of green and yellow vegetables on the back of a truck.
hypothesis: A man looks at a pile of colored vegetables on a car outside.
[n, c, e]: [16.0, 42.0, 42.0]
premise: Multiple people starting to pack their parachutes after a successful skydive.
hypothesis: people going skydiving
[n, c, e]: [14.0, 42.0, 44.0]
premise: A black and white dog running through shallow water.
hypothesis: Two dogs running through water.
[n, c, e]: [5.0, 49.0, 46.0]
premise: Man standing on a tricycle riding in front of a crowd.
hypothesis: A man is on a bike.
[n, c, e]: [4.0, 48.0, 48.0]
premise: A little girl sitting in a small body of water, with sand and beach toys surrounding her.
hypothesis: The girl is seated near water.
